In [49]:
import sys
import random
from pprint import pprint

import pandas as pd
import pyLDAvis

from twitpol import config, language, topic_modelling

if '..' not in sys.path:
    sys.path.append('..')
    
from src.supp_analysis import topic_modelling_and_collocation as tmac

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
nlp = language.get_nlp()

In [3]:
corpus = tmac.load_corpus()

/Users/benjaminlevy/anaconda/envs/twitpol-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(3979785, 4)


In [4]:
corpus.head()

,tweet,nlikes,DateTime,Sentiment
0,"Yes, I agree the workday should be shortened, ...",1.0,2019-11-07,0.503561
1,People think that businesses care more about p...,4.0,2019-11-07,0.740527
2,"@amyklobuchar is rising, surging past Kamala H...",0.0,2019-11-07,0.909588
3,I truly hope so. All this outrage and fake hea...,1.0,2019-11-07,0.222077
4,"She did, and I like Kamala. But @JulianCastro ...",244.0,2019-11-07,0.544374


In [5]:
corpus = tmac.match_to_candidates(corpus)

  0%|          | 0/3979785 [00:00<?, ?it/s]

Checking if tweets match SANDERS


  0%|          | 1367/3979785 [00:00<04:51, 13669.72it/s]

Checking if tweets match HARRIS


  0%|          | 1/3979785 [00:00<118:23:27,  9.34it/s]

...

  0%|          | 1/3979785 [00:00<184:13:43,  6.00it/s]

Checking if tweets match OROURKE


  0%|          | 1/3979785 [00:00<133:17:10,  8.29it/s]

Checking if tweets match WARREN


100%|██████████| 3979785/3979785 [00:08<00:00, 497084.40it/s]


In [6]:
len(corpus[corpus['OROURKE']])

77392

In [7]:
corpus.sample(10)[['tweet', 'SANDERS', 'HARRIS', 'BIDEN', 'BUTTIGIEG', 'BOOKER', 'OROURKE', 'WARREN']]

,tweet,SANDERS,HARRIS,BIDEN,BUTTIGIEG,BOOKER,OROURKE,WARREN
494452,Wow...gutsy. Instead of actually refuting Gabb...,False,True,True,False,False,False,False
195353,Of course. That makes it easier for you to pre...,True,False,False,False,False,False,False
108761,Honestly except for Pete & Bernie & Tulsi I th...,True,False,False,False,False,True,True
752443,Making working class and poor people think tha...,False,False,False,False,False,False,False
616044,Ian as Kamala's campaign tanks. pic.twitter.c...,False,True,False,False,False,False,False
173049,SUPPORT KAMALA HARRIS FOR THE PEOPLE ??? FOR T...,False,False,False,False,False,False,False
121390,Joe Biden never spoke for me in American woman...,False,False,True,False,False,False,False
443811,"Wow Pete can prevent hail, holy shit!",False,False,False,False,False,False,False
94985,After all the high falutin from her about this...,False,False,False,False,False,False,True
746254,I don't say this often but SNL totally nailed ...,False,True,False,False,False,False,False


In [31]:
sanders = corpus[corpus['SANDERS']].sample(10000)
docs = language.make_docs(sanders['tweet'], nlp, notebook=True)

In [39]:
random.choice(docs)

['well',
 'pay',
 'tax',
 'increase',
 'not',
 'good',
 'off',
 '\n\n ',
 'watch',
 '\n\n ',
 'well',
 'nevermind',
 'bernie',
 '0',
 'chance',
 'become',
 'president']

In [40]:
bow, vocab = topic_modelling.make_bow(docs)

In [41]:
model = topic_modelling.run_lda(bow, vocab, num_topics=10)

In [42]:
model.print_topics()

[(0,
  '0.064*"bernie" + 0.045*"  " + 0.031*"sanders" + 0.027*"   " + 0.020*"biden" + 0.019*"warren" + 0.016*"not" + 0.013*"joe" + 0.012*"\n\n " + 0.010*"elizabeth"'),
 (1,
  '0.053*"   " + 0.049*"  " + 0.032*"bernie" + 0.015*"@berniesander" + 0.014*"sanders" + 0.008*"0" + 0.007*"credit" + 0.007*"union" + 0.006*"15" + 0.006*"wage"'),
 (2,
  '0.041*"$" + 0.022*"bernie" + 0.013*"million" + 0.011*"’s" + 0.009*"donation" + 0.008*"child" + 0.006*"\'s" + 0.006*"weekend" + 0.006*"war" + 0.006*"m"'),
 (3,
  '0.048*"pete" + 0.046*"kamala" + 0.041*"\n\n " + 0.035*"harris" + 0.033*"warren" + 0.027*"beto" + 0.023*"buttigieg" + 0.023*"biden" + 0.021*"elizabeth" + 0.019*"mayor"'),
 (4,
  '0.058*"bernie" + 0.046*"not" + 0.014*"get" + 0.013*"\n\n " + 0.012*"go" + 0.011*"like" + 0.011*"people" + 0.010*"would" + 0.010*"want" + 0.009*"  "'),
 (5,
  '0.075*"bernie" + 0.046*"not" + 0.018*"  " + 0.016*"like" + 0.011*"people" + 0.011*"say" + 0.010*"would" + 0.010*"no" + 0.010*"\'s" + 0.009*"support"'),
 (6,


Now for Harris

In [47]:
harris = corpus[corpus['HARRIS']].sample(10000)
docs_harris = language.make_docs(harris['tweet'], nlp, notebook=True)
print(random.choice(docs_harris))
bow_harris, vocab_harris = topic_modelling.make_bow(docs_harris)
model_harris = topic_modelling.run_lda(bow_harris, vocab_harris, num_topics=10)
pprint(model_harris.print_topics())


["c'mon", 'kamala', '  ', 'pic.twitter.com/192lir5fga']
[(0,
  '0.047*"kamala" + 0.041*"harris" + 0.039*"warren" + 0.032*"elizabeth" + '
  '0.030*"biden" + 0.029*"bernie" + 0.027*"pete" + 0.024*"joe" + '
  '0.020*"sanders" + 0.019*"  "'),
 (1,
  '0.058*"kamala" + 0.025*"not" + 0.015*"get" + 0.012*"harris" + 0.010*"like" '
  '+ 0.010*"candidate" + 0.010*"warren" + 0.009*"go" + 0.009*"support" + '
  '0.008*"need"'),
 (2,
  '0.066*"kamala" + 0.059*"not" + 0.023*"would" + 0.018*"like" + 0.017*"think" '
  '+ 0.014*"harris" + 0.014*"biden" + 0.014*"good" + 0.013*"trump" + '
  '0.013*"know"'),
 (3,
  '0.012*"sex" + 0.011*"states" + 0.011*"united" + 0.010*"team" + '
  '0.010*"propose" + 0.010*"=" + 0.008*"defeat" + 0.008*"wake" + 0.006*"knee" '
  '+ 0.006*"crisis"'),
 (4,
  '0.042*"  " + 0.039*"\xa0  " + 0.029*"via" + 0.027*"@kamalaharris" + '
  '0.023*"kamala" + 0.022*"   " + 0.020*"$" + 0.013*"harris" + '
  '0.012*"@youtube" + 0.011*"@ewarren"'),
 (5,
  '0.011*"kamala" + 0.010*"de" + 0.010*

In [50]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(model_harris, corpus_)